In [1]:
pip install httpx BeautifulSoup4 polars tqdm

In [2]:
pip install --upgrade httpx beautifulsoup4 tqdm polars


In [3]:
pip install --upgrade asyncio


In [5]:
import json
import os
import httpx
from bs4 import BeautifulSoup
from tqdm import tqdm
import polars as pl

# Define the base URL and the maximum number of pages for each level
base_url = "https://www.fortiguard.com/encyclopedia?type=ips&risk={level}&page={page}"
max_pages = [5, 10, 15, 20, 25]  # Define the maximum number of pages for each level

def scrape_page(url):
    try:
        response = httpx.get(url)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        title = soup.find('title').text  # Replace with actual HTML elements for title
        link = url  # Replace with actual HTML elements for link
        return {"title": title, "link": link}
    except Exception as e:
        print(f"Error while scraping {url}: {str(e)}")
        return None

def scrape_level(level, max_page):
    scraped_data = []

    for page in range(1, max_page + 1):
        url = base_url.format(level=level, page=page)
        result = scrape_page(url)
        if result:
            scraped_data.append(result)

    return scraped_data

def main():
    dataset_dir = "datasets"
    skipped = {}
    os.makedirs(dataset_dir, exist_ok=True)

    for level, max_page in enumerate(max_pages, start=1):
        print(f"Scraping Level {level}")
        scraped_data = scrape_level(level, max_page)

        if scraped_data:
            df = pl.DataFrame(scraped_data)
            csv_file = os.path.join(dataset_dir, f"forti_lists_{level}.csv")
            df.write_csv(csv_file)

        skipped[level] = [i for i in range(1, max_page + 1) if i not in range(1, len(scraped_data) + 1)]

    with open(os.path.join(dataset_dir, 'skipped.json'), 'w') as json_file:
        json.dump(skipped, json_file)

if __name__ == "__main__":
    main()






Scraping Level 1
Scraping Level 2
Scraping Level 3
Scraping Level 4
Scraping Level 5
